In [1]:
#create axis class
class axis:
    def __init__(s,a,b):
        s.a=min(a,b)
        s.b=max(a,b)
    
    def range(s):
        return s.b-s.a
    
    def touch(s, o):
        if s.a > o.b:
            return False
        if s.b < o.a:
            return False
        return True    
    
    def union(s, o):
        if s.touch(o):
            return axis(min(s.a,o.a),max(s.b,o.b))
        return None
    
    
    
    #this function takes range a and applies range b. 
    #Two lists are returned. 
    #The parts that are stamped out and those left untouched.
    def stamp(s,o):
        hit   = []
        fre  = []  
        if s.touch(o):
            #####AAAAA#####
            #####BBBBB#####
            ###############
            if s.a >= o.a and s.b <= o.b:
                print("case A")
                hit.append(axis(s.a, s.b))
            ####AAAAA######
            #####BBBBB#####
            ####A##########
            elif s.a < o.a and s.b <= o.b:
                print("case B")
                fre.append(axis(s.a, o.a-1))
                hit.append(axis(o.a, s.b))
            ######AAAAA######
            #####BBBBB#######
            ##########A######
            elif s.a >= o.a and s.b > o.b:
                print("case C")
                hit.append(axis(s.a, o.b))
                fre.append(axis(o.b+1, s.b))
            ######AAAAA######
            ########BB#######
            ######AA##A######
            elif s.a < o.a and s.b > o.b:
                print("case D")
                fre.append(axis(s.a, o.a-1))
                hit.append(axis(o.a, o.b))
                fre.append(axis(o.b+1, s.b))
            else:
                print("unhandled case")
                assert(0)            
        else:
            print("case untouched")
            fre.append(a)
        return (hit, fre)
    
    def print(s, c):
        p = ""
        for i in range(20):
            if i >= s.a and i <= s.b:
                p += c
            else:
                p += " "
        print(p)
    

tests = []
tests.append((axis(2,7),axis(2,7)))
tests.append((axis(3,9),axis(2,7)))
tests.append((axis(3,5),axis(2,7)))
tests.append((axis(2,7),axis(3,5)))
tests.append((axis(8,9),axis(2,7)))

for a,b in tests:
    a.print("A")
    b.print("B")
    c0,c1 = a.stamp(b)
    for c in c0:
        c.print("C")
    for c in c1:
        c.print("c")
    print()


  AAAAAA            
  BBBBBB            
case A
  CCCCCC            

   AAAAAAA          
  BBBBBB            
case C
   CCCCC            
        cc          

   AAA              
  BBBBBB            
case A
   CCC              

  AAAAAA            
   BBB              
case D
   CCC              
  c                 
      cc            

        AA          
  BBBBBB            
case untouched
        cc          



In [11]:
#create rect class
class rect:
    def __init__(s,x0,x1,y0,y1):
        s.x = axis(x0,x1)
        s.y = axis(y0,y1)
    
    def draw(s, fb, c):
        for y in range(s.y.a,s.y.b+1, 1):
            for x in range(s.x.a,s.x.b+1, 1):
                fb[y][x] = c
    
    def stamp(s, o):
        hit = []
        fre = []
        x_h,x_f = s.x.stamp(o.x)
        y_h,y_f = s.y.stamp(o.y)
        
        return hit,fre
        


    
tests = []
tests.append((rect(2, 8, 2, 8), rect(4, 12, 3, 7)))

for r0, r1 in tests:
    fb = []
    for y in range(20):
        l = []
        for x in range(30):
            l.append('.')
        fb.append(l)
    
    r0.draw(fb, "A")
    r1.draw(fb, "B")
    
    #hit,fre = r0.stamp(r1)
    #for i in range(len(fre)):
    #    hit[i].draw(fb, chr(ord('a') + i))
    
    #for i in range(len(hit)):
    #    hit[i].draw(fb, chr(ord('A') + i))
        
        
    
    
    for l in fb:
        print("".join(l))



..............................
..............................
..AAAAAAA.....................
..AABBBBBBBBB.................
..AABBBBBBBBB.................
..AABBBBBBBBB.................
..AABBBBBBBBB.................
..AABBBBBBBBB.................
..AAAAAAA.....................
..............................
..............................
..............................
..............................
..............................
..............................
..............................
..............................
..............................
..............................
..............................


In [ ]:

on=[]
x = []
y = []
cmds = []
for l in open("06.txt").readlines():
    l=l.strip().split(" ")
    op=l[-4][:2]
    x0,y0=[int(x) for x in l[-3].split(",")]
    x1,y1=[int(x) for x in l[-1].split(",")]
    cmds.append((op, x0,y0,x1,y1))

off=[rect(0,999,0,999)]
on=[]
for op, x0, y0, x1, y1 in cmds:
    if op == "on":
        pass
    if op == "of":
        pass
    if op == "to":
        pass
 